## Introduction

- Tao đang kiểm tra nhẹ làm thế nào để run một chiếc spark load tensorflow model nha mấy bé. 

- mấy bé thấy hay thì nhớ lài lái lai và sè sé se. và follow github cho tao nha: [Github lè](https://github.com/dnguyenngoc) or [Github group lè](https://github.com/apot-group)

<div class="alert alert-block alert-info">
<b>Step 1:</b> Import Libraries
</div>

In [1]:
# Spark Session, Pipeline, Functions, and Metrics
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession
from pyspark.ml import Pipeline
import pandas as pd
import os

os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.2,' \
                                    + 'org.apache.spark:spark-streaming-kafka-0-10_2.12:3.0.2,' \
                                    + 'org.apache.kafka:kafka-clients:3.1.0,org.apache.spark:spark-core_2.12:3.0.2,' \
                                    + 'org.apache.spark:spark-streaming_2.12:3.0.2,' \
                                    + 'org.apache.spark:spark-token-provider-kafka-0-10_2.12:3.0.2,' \
                                    + 'org.apache.hadoop:hadoop-core:1.2.1' \
                                    + ' pyspark-shell'

<div class="alert alert-block alert-info">
<b>Step 2:</b> Start Spark Session
</div>

In [3]:
ss  = SparkSession.builder.master('local[1]') \
                  .appName("test") \
                  .config("spark.executor.memory", "2g") \
                  .config("spark.driver.memory", "2g") \
                  .config("spark.executor.cores", "2") \
                  .config("spark.driver.cores", "2") \
                  .getOrCreate()
sc = ss.sparkContext
sc

<SparkContext master=local[1] appName=test>

<div class="alert alert-block alert-info">
<b>Step 3:</b> Load tensoflow model
</div>

In [38]:
pandas_df = pd.read_csv(
     "https://github.com/bgweber/Twitch/raw/master/Recommendations/games-expand.csv")
pandas_df

,G1,G2,G3,G4,G5,G6,G7,G8,G9,G10,label
0,0,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,0
3,0,0,1,0,0,1,1,0,0,1,1
4,0,0,1,0,1,1,0,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...
22900,0,0,1,0,0,0,0,0,0,0,0
22901,1,0,0,0,0,0,0,0,0,0,0
22902,0,0,0,0,0,0,0,0,0,0,1
22903,0,0,1,0,0,0,0,0,0,0,0


In [41]:
spark_df = ss.createDataFrame(pandas_df)

# assign a user ID and a partition ID using Spark SQL
spark_df.createOrReplaceTempView("spark_df")
spark_df = ss.sql("""
select *, user_id%10 as partition_id 
from (
  select *, row_number() over (order by rand()) as user_id
  from spark_df
) 
""")

# preview the results
spark_df.show()

22/03/02 16:27:11 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+---+---+---+---+---+---+---+---+---+---+-----+-------+------------+
| G1| G2| G3| G4| G5| G6| G7| G8| G9|G10|label|user_id|partition_id|
+---+---+---+---+---+---+---+---+---+---+-----+-------+------------+
|  0|  0|  0|  0|  0|  0|  0|  1|  0|  0|    0|      1|           1|
|  0|  0|  1|  0|  0|  0|  0|  0|  0|  0|    0|      2|           2|
|  0|  0|  0|  0|  0|  0|  1|  1|  0|  0|    0|      3|           3|
|  0|  0|  0|  0|  0|  0|  1|  0|  0|  0|    0|      4|           4|
|  0|  0|  0|  0|  1|  0|  0|  0|  0|  0|    0|      5|           5|
|  0|  0|  0|  0|  1|  0|  0|  1|  0|  0|    0|      6|           6|
|  0|  0|  0|  0|  0|  0|  0|  0|  1|  0|    0|      7|           7|
|  0|  0|  0|  0|  0|  0|  1|  0|  0|  0|    0|      8|           8|
|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|    0|      9|           9|
|  0|  0|  0|  0|  0|  0|  1|  1|  0|  0|    1|     10|           0|
|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|    1|     11|           1|
|  0|  0|  0|  0|  1|  0|  0|  0| 

In [56]:
from sklearn.linear_model import LogisticRegression

df = spark_df.toPandas().drop(['user_id', 'partition_id'], axis = 1)

y_train = df['label']
x_train = df.drop(['label'], axis=1)

# use logistic regression
model = LogisticRegression()
model.fit(x_train, y_train)

LogisticRegression()

In [57]:
# pull all data to the driver node
sample_df = spark_df.toPandas()

# create a prediction for each user 
ids = sample_df['user_id']
x_train = sample_df.drop(['label', 'user_id', 'partition_id'], axis=1)
pred = model.predict_proba(x_train)
result_df = pd.DataFrame({'user_id': ids, 'prediction': pred[:,1]})

# display the results 
e = ss.createDataFrame(result_df)
e.show()

+-------+--------------------+
|user_id|          prediction|
+-------+--------------------+
|      1| 0.06414947633646334|
|      2| 0.14233082944037895|
|      3|0.045371032095244526|
|      4| 0.07175760020956665|
|      5|  0.1350842735918626|
|      6| 0.08760914967461457|
|      7| 0.21250043631356066|
|      8| 0.07175760020956665|
|      9| 0.10030953139889025|
|     10|0.045371032095244526|
|     11| 0.10030953139889025|
|     12|  0.1350842735918626|
|     13|  0.3809000596496006|
|     14|  0.1350842735918626|
|     15| 0.05527800275089643|
|     16| 0.06414947633646334|
|     17| 0.06414947633646334|
|     18| 0.03229025205257302|
|     19|0.045599767602852205|
|     20| 0.07175760020956665|
+-------+--------------------+
only showing top 20 rows



In [58]:
df = ss.read.format("image").load('/work/data/images/').limit(100)

In [59]:
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.types import *

# define a schema for the result set, the user ID and model prediction
schema = StructType([StructField('user_id', LongType(), True),
                     StructField('prediction', DoubleType(), True)])  

# define the Pandas UDF 
@pandas_udf(schema, PandasUDFType.GROUPED_MAP)
def apply_model(sample_pd):

    # run the model on the partitioned data set 
    ids = sample_df['user_id']
    x_train = sample_df.drop(['label', 'user_id', 'partition_id'], axis=1)
    pred = model.predict_proba(x_train)

    return pd.DataFrame({'user_id': ids, 'prediction': pred[:,1]})
  
# partition the data and run the UDF  
results = spark_df.groupby('partition_id').apply(apply_model)
results.show()

/usr/local/lib/python3.8/site-packages/pyspark/sql/pandas/group_ops.py:81: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(
22/03/02 16:36:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-------+--------------------+
|user_id|          prediction|
+-------+--------------------+
|      1| 0.06414947633646334|
|      2| 0.14233082944037895|
|      3|0.045371032095244526|
|      4| 0.07175760020956665|
|      5|  0.1350842735918626|
|      6| 0.08760914967461457|
|      7| 0.21250043631356066|
|      8| 0.07175760020956665|
|      9| 0.10030953139889025|
|     10|0.045371032095244526|
|     11| 0.10030953139889025|
|     12|  0.1350842735918626|
|     13|  0.3809000596496006|
|     14|  0.1350842735918626|
|     15| 0.05527800275089643|
|     16| 0.06414947633646334|
|     17| 0.06414947633646334|
|     18| 0.03229025205257302|
|     19|0.045599767602852205|
|     20| 0.07175760020956665|
+-------+--------------------+
only showing top 20 rows

